In [116]:
#import libraries
import requests
import pandas as pd
import time
import requests

In [122]:
#Keys
API_KEY = ""


In [123]:
def get_channels(SEARCH_QUERY, MAX_RESULTS):
    channels = []
    next_page_token = None

    while len(channels) < MAX_RESULTS:
        url = f'https://www.googleapis.com/youtube/v3/search?key={API_KEY}&q={SEARCH_QUERY}&type=channel&part=id&maxResults={MAX_RESULTS}&pageToken={next_page_token}' if next_page_token else f'https://www.googleapis.com/youtube/v3/search?key={API_KEY}&q={SEARCH_QUERY}&type=channel&part=id&maxResults={MAX_RESULTS}'

        try:
            response = requests.get(url)
            response.raise_for_status()  # Check for HTTP errors

            data = response.json()

            if 'items' in data:
                channels.extend([item['id']['channelId'] for item in data['items']])

            if 'nextPageToken' in data:
                next_page_token = data['nextPageToken']
            else:
                break

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            break

    #print(f"Total channels retrieved: {len(channels)}")
    #print(channels)
    return channels

In [124]:

def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    #view_count = response_video_stats['items'][0]['statistics']['viewCount']
    #like_count = response_video_stats['items'][0]['statistics']['likeCount']
    
    # Check if 'dislikeCount' key is present in the response, otherwise set to 0
    if 'dislikeCount' in response_video_stats['items'][0]['statistics']:
        dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    else:
        dislike_count = 0
        
    # Check if 'commentcount' key is present in the response, otherwise set to 0
    if 'commentCount' in response_video_stats['items'][0]['statistics']:
         comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    else:
        comment_count = 0
        
        # Check if 'commentcount' key is present in the response, otherwise set to 0
    if 'likeCount' in response_video_stats['items'][0]['statistics']:
         like_count = response_video_stats['items'][0]['statistics']['likeCount']
    else:
        like_count = 0       
        
        # Check if 'commentcount' key is present in the response, otherwise set to 0
    if 'viewCount' in response_video_stats['items'][0]['statistics']:
         view_count = response_video_stats['items'][0]['statistics']['viewCount']
    else:
        like_count = 0      
    #comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, dislike_count, comment_count
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    #comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    #return view_count, like_count, dislike_count, comment_count 

In [125]:
def get_videos(df, CHANNEL_ID, MAX_RESULTS):
    pageToken = ""
    all_data = []  # Initialize a list to store video data
    
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults="+str(MAX_RESULTS)+"&"+pageToken

    response = requests.get(url).json()
    time.sleep(1)  # Give it a second before starting the for loop

    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&", "")
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]
            view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

            video_data = {
                'channel_id': CHANNEL_ID,
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': view_count,
                'like_count': like_count,
                'dislike_count': dislike_count,
                'comment_count': comment_count
            }
            all_data.append(video_data)
                
    new_df = pd.DataFrame(all_data)  # Create a DataFrame from the collected data
    df = pd.concat([df, new_df], ignore_index=True)  # Concatenate the new data with the existing DataFrame
    
    return df


In [126]:
#main

search_queries = ['arsenal', 'barcelona', 'chelsea']

for SEARCH_QUERY in search_queries:
    #build our dataframe
    df2 = pd.DataFrame(columns=["channel_id","video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"]) 

    channels = get_channels(SEARCH_QUERY, 1)

    for channel_id in channels:
        df2 = get_videos(df2, channel_id, 5)
    
    search_query_name = SEARCH_QUERY.replace(" ", "")
    df2.to_csv(f'youtube_vids_{search_query_name}.csv')